<a href="https://colab.research.google.com/github/anosharahim/deep-learning/blob/master/DL_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Packages and Libraries

In [6]:
import os
import shutil
import math 
import random 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
import tensorflow as tf

import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.layers.pooling import GlobalAveragePooling2D
from keras.models import Sequential, Model 
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import batch_normalization
from tensorflow.keras.optimizers import RMSprop, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau 

## Load Data

In [7]:
#load data from gdrive
train_npz = np.load("/content/drive/MyDrive/Capstone/datasets/train.npz")
test_npz = np.load("/content/drive/MyDrive/Capstone/datasets/test.npz")

x_train = train_npz["arr_0"]
y_train = train_npz["arr_1"]
x_test = test_npz["arr_0"]
y_test = test_npz["arr_1"]

#add 2 more channels to grayscale image to imitate rgb
x_train = np.repeat(x_train[..., np.newaxis], 3, -1)
x_test = np.repeat(x_test[..., np.newaxis], 3, -1)

#split test set into validation and test set 
x_val, x_test, y_val ,y_test = train_test_split(x_test,y_test, test_size=0.3)

print('Train -- ', x_train.shape, y_train.shape)
print('Val -- ', x_val.shape, y_val.shape)
print('Test -- ', x_test.shape, y_test.shape)

Train --  (28709, 48, 48, 3) (28709,)
Val --  (5024, 48, 48, 3) (5024,)
Test --  (2154, 48, 48, 3) (2154,)


## Prepare Model 

In [8]:
vgg = VGG16(weights = 'imagenet', include_top=False, input_shape= (48,48,3))

for layer in vgg.layers:
  layer.trainable=False

for (i,layer) in enumerate(vgg.layers):
  print(str(i)+' '+ layer.__class__.__name__, layer.trainable)

def fully_connected(base_model, num_classes):
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(1024, activation='relu')(x)
  x = Dense(1024, activation='relu')(x)
  x = Dense(1024, activation='relu')(x)
  x = Dense(512, activation='relu')(x)
  x = Dense(num_classes, activation='softmax')(x)
  return x

num_class = 7
FC_head = fully_connected(vgg, num_class)
model = Model(inputs=vgg.input, outputs = FC_head)
print(model.summary())

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                             

## Model Training 

Serious overfitting rn

In [9]:
epochs = 100
bs = 32
rms_lr = 0.001
sgd_lr = 0.01
sgd_decay = 0.0001

lrd = ReduceLROnPlateau(monitor='val_accuracy',mode='max',factor=0.3, patience=10, min_lr=0.00001, verbose=1)
es = EarlyStopping(monitor = 'val_accuracy',min_delta = 0, patience = 10, verbose = 1, restore_best_weights = True)
checkpoint = ModelCheckpoint('face_vgg.h5', monitor = 'val_loss', mode ='min', save_best_only = True, verbose=1)
callbacks = [checkpoint, es, lrd]

sgd = SGD(lr=sgd_lr, momentum=0.9, decay=sgd_decay, nesterov=True) 
rms = RMSprop(lr=rms_lr)

model.compile(loss = 'sparse_categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

history = model.fit(x_train, y_train, 
                    batch_size=bs, 
                    epochs=epochs, 
                    verbose=1,
                    validation_data= (x_val, y_val),
                    callbacks=callbacks)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/100
897/898 [============================>.] - ETA: 0s - loss: 2.0093 - accuracy: 0.3382
Epoch 1: val_loss improved from inf to 1.62910, saving model to face_vgg.h5
898/898 [==============================] - 16s 18ms/step - loss: 2.0092 - accuracy: 0.3382 - val_loss: 1.6291 - val_accuracy: 0.3770 - lr: 0.0100
Epoch 2/100
898/898 [==============================] - ETA: 0s - loss: 1.6003 - accuracy: 0.3766
Epoch 2: val_loss improved from 1.62910 to 1.60984, saving model to face_vgg.h5
898/898 [==============================] - 16s 18ms/step - loss: 1.6003 - accuracy: 0.3766 - val_loss: 1.6098 - val_accuracy: 0.3812 - lr: 0.0100
Epoch 3/100
897/898 [============================>.] - ETA: 0s - loss: 1.5636 - accuracy: 0.3919
Epoch 3: val_loss improved from 1.60984 to 1.58345, saving model to face_vgg.h5
898/898 [==============================] - 16s 17ms/step - loss: 1.5637 - accuracy: 0.3919 - val_loss: 1.5835 - val_accuracy: 0.3796 - lr: 0.0100
Epoch 4/100
897/898 [==============